In [1]:
import os
import pandas as pd

import re
import time
import random
import string
from typing import Any

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
RANDOM_SEED = 42
ROOT_DIR = '../predictions/'

In [3]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
seed_all(RANDOM_SEED)

In [6]:
embeddings = pd.read_csv(os.path.join(ROOT_DIR, 'embeddings_submission.csv'))
naive_bayes_simple = pd.read_csv(os.path.join(ROOT_DIR, 'naive_bayes_simple_all_level.csv'))
naive_bayes_laplace = pd.read_csv(os.path.join(ROOT_DIR, 'naive_bayes_laplace_all_level.csv'))

final = pd.concat([embeddings, naive_bayes_simple, naive_bayes_laplace], axis=1)

In [7]:
final.head()

,MessageId,Suspicious_Level,MessageId,NaiveBayesSimple_Suspicious_Level_0,NaiveBayesSimple_Suspicious_Level_1,NaiveBayesSimple_Suspicious_Level_2,MessageId,NaiveBayes_Suspicious_Level_0,NaiveBayes_Suspicious_Level_1,NaiveBayes_Suspicious_Level_2
0,168641,3,168641,0.594159,0.199875,-3.354664,168641,0.004438,0.003675,0.000213
1,36569,1,36569,2.180644,-2.523844,-0.721312,36569,0.017506,0.000243,0.000552
2,19697,1,19697,2.321289,-4.937329,-4.238373,19697,0.998395,0.000428,0.000687
3,49296,3,49296,0.075663,-1.904409,-1.538347,49296,0.119164,0.008813,0.013954
4,11784,1,11784,3.333188,-3.221182,-3.515524,11784,0.492804,0.000353,0.000346
